<a href="https://colab.research.google.com/github/msorrentino3/Tirocinio/blob/main/AiAgentSingleArchitetture.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **DistillBERT Model**







> importo librerie



In [ ]:
import torch
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification
from torch.utils.data import TensorDataset, DataLoader




> Lettura DataSet e trasformazione dei dati in formato String per essere poi suddivisi in maniera randomica per l'adddestramento del modello



In [ ]:
#caricamento del dataset
data = pd.read_csv("data.csv")
#creo nuova colonna nel dataset dove metto 1 se nella colonna 3 compare attack altrimenti metto 0
data["label"] = np.where(data["class3"] == "Attack", 1, 0)

#creo testo per addestare modello distillbert
#converto ogni colonna nel formato string e se nella cella della colonna del dataset risulta
#senza dato verrà sostituito con stringa vuota
scr = data["Scr_IP"].astype(str).replace("nan", "")
des = data["Des_IP"].astype(str).replace("nan", "")
proto = data["Protocol"].astype(str).replace("nan", "")
serv = data["Service"].astype(str).replace("nan", "")
alert = data["OSSEC_alert"].astype(str).replace("nan", "")

#creo nel dataset colonna nella quale metto sottoforma di string il contenuto della riga
data["text"] = pd.Series(
    np.where(scr != "", "Da " + scr + " ", "") +
    np.where(des != "", "A " + des + " ", "") +
    np.where(proto != "", "Protocollo " + proto + " ", "") +
    np.where(serv != "", "Servizio " + serv + " ", "") +
    np.where((alert != "") & (alert != "0"), "Alert " + alert, "")
).str.strip()

#divido il set di dati per addestrare il modello (60% dei dati) e sia per testarlo(40%)
train_df, test_df = train_test_split(
    data[["text", "label"]],
    test_size=0.4,
    stratify=data["label"],
    random_state=42
)
data = pd.read_csv("data.csv")



> Tokenizzazzione dati sia per l'addestramento e sia per la valutazione del modello






In [ ]:
#tokenizzo sia i dati di addestramento che quelli di test
tokenizer = DistilBertTokenizerFast.from_pretrained("distilbert-base-uncased")

train_enc = tokenizer(list(train_df["text"]), padding=True, truncation=True, max_length=128, return_tensors="pt")
test_enc  = tokenizer(list(test_df["text"]),  padding=True, truncation=True, max_length=128, return_tensors="pt")

# Aggiungo le etichette
train_enc['labels'] = torch.tensor(train_df["label"].values)
test_enc['labels']  = torch.tensor(test_df["label"].values)

train_dataset = TensorDataset(train_enc['input_ids'], train_enc['attention_mask'], train_enc['labels'])
test_dataset  = TensorDataset(test_enc['input_ids'],  test_enc['attention_mask'],  test_enc['labels'])

#creazione dataset e dataloder
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, pin_memory=True, num_workers=4)
test_loader  = DataLoader(test_dataset,  batch_size=32, shuffle=False, pin_memory=True, num_workers=4)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(




> Selezione disposito GPU o CPU e creazione modello DistillBERT



In [ ]:
#seleziono se utilizzare la gpu se è libera oppure presente nel dispositivo altrimenti utilizza la gpu
device = "cuda" if torch.cuda.is_available() else "cpu"

#scarico modello pre-addestrato di ditillbert ingrado di fare delle classificazioni
model = DistilBertForSequenceClassification.from_pretrained(
    "distilbert-base-uncased",
    num_labels=2 #utilizzo lables = 2 perchè devo fare due tipi di classificazione (attacco e normale)
).to(device)

optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5) #ottimizzo i parametri e passo la velocità con cui il modello impara


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.




> Addestramento del modello



In [ ]:
#Addestro il modello
model.train()
epoche = 3 #scelto 3 perchè è così ho prestazioni superiori al 90% anche
for ep in range(epoche): #il modello scansiona i dati per essere addestrato x il numero di epoche
    tot_loss = 0
    for input_ids, attention_mask, labels in train_loader: #prendo i dati per l'allenamento
        input_ids = input_ids.to(device, non_blocking=True) #sposto il testo tokenizzato sulla gpu per rendere più veloce l'esecuzione
        attention_mask = attention_mask.to(device, non_blocking=True) #spsoto nella gpu sullo le parti di testo che non sono rimpieti con il padding
        labels = labels.to(device, non_blocking=True) #porto sulla gpu i valori per indicare se si tratta di un attacco o meno
        optimizer.zero_grad() #azzero i parametri così nella prossima epoca i valori non si sommino
        loss = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels).loss #il modello legge il testo e cerca di capire se si tratta di un attacco o meno
        #infine calcola quanto ha sbagliato
        loss.backward() #calcola di quanto devono essere aggiornati i pesi dei parametri per la prossima epoca
        optimizer.step()#cambio i pesi dei parametri (il modello ha "imparato")

        tot_loss += loss.item() #calcolo la media dell'errore alla fine dell'epoca
    print(f"Epoca {ep+1}: Loss media = {tot_loss/len(train_loader):.4f}")


Epoca 1: Loss media = 0.5543
Epoca 2: Loss media = 0.2370
Epoca 3: Loss media = 0.1689




> Valutazione del modello



In [ ]:
#valuto il modello
model.eval() #setto il modello in mdalità valutazione così che non aggiorni più i pesi dei parametri
all_preds = []
all_labels = []

with torch.no_grad():
    for input_ids, attention_mask, labels in test_loader:#prendo i dati per effettuare il test del modello
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        labels = labels.to(device)

        logits = model(input_ids=input_ids, attention_mask=attention_mask).logits #il modello effettua la previsione
        preds = logits.argmax(dim=1) #sceglie se si tratta di un attacco oppure no

        all_preds.extend(preds.cpu().numpy()) #salva le predizioni e le sposta sulla cpu in formato numpy
        all_labels.extend(labels.cpu().numpy())#salva i dati del batch e li sposta sulla cpu in formato numpy

# Calcolo metriche
accuracy = accuracy_score(all_labels, all_preds)
precision = precision_score(all_labels, all_preds)
recall = recall_score(all_labels, all_preds)
f1 = f1_score(all_labels, all_preds)

print(f"\nAccuracy:  {accuracy*100:.2f}%")
print(f"Precision: {precision*100:.2f}%")
print(f"Recall:    {recall*100:.2f}%")
print(f"F1-score:  {f1*100:.2f}%")


Accuracy:  93.25%
Precision: 93.37%
Recall:    92.89%
F1-score:  93.13%


#  **AI Agent**



> Creazione classe AI Agent



In [ ]:
#creo classe ai agent
class IntrusionDetectionAgent:

  #costruttore della classe nella quale passo il modello DistillBERT addestrato precedentemente,
  #il tokenizer di DistillBert con il quale posso leggere i pacchetti di rete e trasformarli in numeri in modo tale che il modello possa leggerli,
  #il divice ovvero cpu o gpu in base a quale sia libero
  #setto il threshould a 0.85 ovvero tutti i pacchetti che vengono classificati come attacco con percentuale 85% o superiore vengono considerati attacchi
    def __init__(self, model, tokenizer, device, threshold=0.85):
        self.model = model
        self.tokenizer = tokenizer
        self.device = device
        self.threshold = threshold

    #funzione che serve ad analizzare i pacchetti ed effettuare operazione nel caso in cui sia un attacco
    #text contiene i pacchetti di rete
    def predict_action(self, text):
        #Tokenizzazione  del/dei pacchetti e invio al device
        inputs = self.tokenizer(
            text,
            return_tensors="pt",
            truncation=True,
            max_length=128
        ).to(self.device)

        #Predizione del pacchetto
        with torch.no_grad():                       #no_grad() disattiva il gradiente per mettere il modello DistillBert in modalità valutazine
            logits = self.model(**inputs).logits    #il modello classifica il pacchetto come attacco(1) oppure come normale(0)
            probs = torch.softmax(logits, dim=1)    #restituisce la probabilità ceh il/i pacchetto/i sia un attacco

        #Etichetta e confidenza
        label = probs.argmax().item()
        confidence = float(probs[0][label])

        #Decisione
        if label == 1 and confidence >= self.threshold:   #Contralla se il pacchetto è stato classificato come attacco e la confidenzialità de l pacchetto
            return "BLOCK", confidence                    #paccehtto viene bloccatto in caso sia classificato come attacco e la confidenza è maggiore o uguale 85%
        elif label == 1:
            return "ALERT", confidence                    #viene inviato un alert in caso sia classificato come attacco e la confidenza è minore 85%
        else:
            return "ALLOW", confidence                    #il paccehtto viene lasciato passare





> Esempio funzionamento



In [ ]:
#esempio funzinamento ipotetico
agent = IntrusionDetectionAgent(model, tokenizer, device)

logs = [
    "Da 192.168.2.199 A 192.168.2.10 Protocollo tcp Servizio http Alert TRUE",
    "Da 10.0.1.5 A 131.236.3.92 Protocollo udp Servizio dns",
    "Da 172.24.1.80 A 172.24.1.1 Protocollo udp Servizio dns",
]

for i, log in enumerate(logs, 1):
    action, conf = agent.predict_action(log)
    print(f"\n📡 LOG {i}")
    print(f"Azione: {action} | Confidenza: {conf:.2f}")


📡 LOG 1
Azione: BLOCK | Confidenza: 0.98

📡 LOG 2
Azione: ALLOW | Confidenza: 0.98

📡 LOG 3
Azione: ALLOW | Confidenza: 0.98
